<a href="https://colab.research.google.com/github/drabhishekk14/AIML_Notebooks/blob/main/Exercise_with_Chris_on_Spark_Sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=570910ac5f807d5c857f8d45ec1693b73d5eda528fe88dc5ce414ed1b729b584
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [3]:
import wget

In [4]:
#update local version of the package catalog
!apt-get update
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark 3.1.3
!wget -q http://apache.osuosl.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.1.3-bin-hadoop3.2.tgz
# install findspark 
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop3.2"

import findspark
findspark.init() 

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [92.1 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.co

In [5]:
from pyspark.sql import SparkSession

https://github.com/tulip-lab/sit742/tree/develop/Jupyter/data


In [6]:
import pandas as pd
df = pd.read_parquet('/content/bank.parquet')


In [7]:
df.head(6)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
5,42,management,single,tertiary,no,0,yes,yes,unknown,5,may,562,2,-1,0,unknown,yes


In [8]:
spark = SparkSession.builder.appName('ml-bank').getOrCreate()
df = spark.read.parquet('/content/bank.parquet') 
df.printSchema() 
df.show(5)


root
 |-- age: long (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: long (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: long (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: long (nullable = true)
 |-- campaign: long (nullable = true)
 |-- pdays: long (nullable = true)
 |-- previous: long (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)

+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|       job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+----

In [10]:
df.show

<bound method DataFrame.show of DataFrame[age: bigint, job: string, marital: string, education: string, default: string, balance: bigint, housing: string, loan: string, contact: string, day: bigint, month: string, duration: bigint, campaign: bigint, pdays: bigint, previous: bigint, poutcome: string, deposit: string]>

In [18]:
df.where(df['age'] > 40)[['age']].show(10) #

+---+
|age|
+---+
| 59|
| 56|
| 41|
| 55|
| 54|
| 42|
| 56|
| 60|
| 46|
| 49|
+---+
only showing top 10 rows



In [21]:
df.where(df['housing']!= 'yes')[['age']].describe().show() #df.where(~df['housing'].like('yes'))[['age']].describe().show()

+-------+-----------------+
|summary|              age|
+-------+-----------------+
|  count|             5881|
|   mean|43.13637136541404|
| stddev|13.62638391002638|
|    min|               18|
|    max|               95|
+-------+-----------------+



In [22]:
df.where(~df['housing'].like('yes'))[['age']].describe().show() # using like ensure any variation of "Yes" to be/
# considered in the filtering

+-------+-----------------+
|summary|              age|
+-------+-----------------+
|  count|             5881|
|   mean|43.13637136541404|
| stddev|13.62638391002638|
|    min|               18|
|    max|               95|
+-------+-----------------+



In [42]:
min_val = df.agg({"age":"min"}).collect()[0][0]
max_val = df.agg({"age":"max"}).collect()[0][0]

#.collect method allows to get the min and max value and store in the variables. The location is given as the min and max is the one value
# and its location is 0 row index and 0 column index

In [29]:
max_age = df.agg({"age":"max"})

In [40]:
scaled_age = df.withColumn('scaled_age', (df['age'] - min_val)/((max_val)-(min_val)))

In [41]:
scaled_age.show()

+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+-------------------+
|age|        job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|         scaled_age|
+---+-----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+-------------------+
| 59|     admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes| 0.5324675324675324|
| 56|     admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes| 0.4935064935064935|
| 41| technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes| 0.2987012987012987|
| 55|   services| married|secondary|     no|   2476|    yes|  no|unknown|  5

In [45]:
def min_max_scaler (df, cols_to_scale):
  for col in cols_to_scale:
    min_val = df.agg({col:"min"}).collect()[0][0]
    max_val = df.agg({col:"max"}).collect()[0][0]
    new_column_name = 'sacled_'+ col

    df.withColumn(new_column_name,(df[col]-min_val)/(max_val - min_val))

  return df

In [46]:
cols_to_scale = ['balance','duration']
min_max_scaler(df,cols_to_scale)[['balance','duration','scaled_balance','scaled_duration']].show()

AnalysisException: ignored